In [1]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from glob import glob

In [3]:
# Paths to the dataset
DATASET_PATH = "/kaggle/input/brain-tumor-detection-mri/Brain_Tumor_Detection"
NO_PATH = os.path.join(DATASET_PATH, "no")
YES_PATH = os.path.join(DATASET_PATH, "yes")

In [10]:
# Parameters
IMG_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 5
LR = 0.01

In [5]:
# Helper function to load data and labels
def load_data_and_labels():
    images = []
    labels = []
    
    # Load images from 'no' and 'yes' folders
    for label, folder in enumerate([NO_PATH, YES_PATH]):
        for file in glob(os.path.join(folder, "*.jpg")):
            img = load_img(file, target_size=(IMG_SIZE, IMG_SIZE))
            img = img_to_array(img) / 255.0
            images.append(img)
            labels.append(label)
    
    return np.array(images), np.array(labels)

In [6]:
# Load data and split into train, validation, and test sets
X, y = load_data_and_labels()
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

In [7]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)
val_generator = ImageDataGenerator().flow(X_val, y_val, batch_size=BATCH_SIZE)
test_generator = ImageDataGenerator().flow(X_test, y_test, batch_size=BATCH_SIZE)

In [8]:
# Model Builder Function
def build_model(base_model):
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')  # Binary classification
    ])
    return model

In [9]:
# DenseNet-121 Model
densenet_base = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
densenet_model = build_model(densenet_base)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [11]:
# Xception Model
xception_base = Xception(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
xception_model = build_model(xception_base)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [12]:
# Compile Models
optimizer = SGD(learning_rate=LR, momentum=0.9)
densenet_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
xception_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model_densenet.keras", save_best_only=True, monitor='val_loss', mode='min')
]

In [15]:
# Train DenseNet-121
print("Training DenseNet-121...")
densenet_history = densenet_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)

Training DenseNet-121...
Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


66/66 ━━━━━━━━━━━━━━━━━━━━ 245s 2s/step - accuracy: 0.7971 - loss: 0.3855 - val_accuracy: 0.5267 - val_loss: 7.1820
Epoch 2/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 30s 412ms/step - accuracy: 0.9618 - loss: 0.1076 - val_accuracy: 0.8822 - val_loss: 0.5103
Epoch 3/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 30s 415ms/step - accuracy: 0.9811 - loss: 0.0610 - val_accuracy: 0.9867 - val_loss: 0.0414
Epoch 4/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 29s 402ms/step - accuracy: 0.9915 - loss: 0.0278 - val_accuracy: 0.9689 - val_loss: 0.0745
Epoch 5/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 29s 400ms/step - accuracy: 0.9923 - loss: 0.0283 - val_accuracy: 0.9800 - val_loss: 0.0576


In [16]:
# Save DenseNet-121 Model
densenet_model.save("densenet121_model.keras")
print("DenseNet-121 training completed and model saved.")

DenseNet-121 training completed and model saved.


In [18]:
# Evaluate on Test Data
print("Evaluating DenseNet-121 on test data...")
densenet_eval = densenet_model.evaluate(test_generator)
print(f"DenseNet-121 Test Accuracy: {densenet_eval[1] * 100:.2f}%")

Evaluating DenseNet-121 on test data...
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.9671 - loss: 0.0734
DenseNet-121 Test Accuracy: 97.33%


In [24]:
# Reinitialize optimizer for Xception
xception_optimizer = SGD(learning_rate=LR, momentum=0.9)
xception_model.compile(optimizer=xception_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Update ModelCheckpoint for Xception
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model_xception.keras", save_best_only=True, monitor='val_loss', mode='min')
]

In [25]:
# Train Xception
print("Training Xception...")
xception_history = xception_model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)

Training Xception...
Epoch 1/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.7026 - loss: 0.5405 - val_accuracy: 0.9600 - val_loss: 0.1078
Epoch 2/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 42s 593ms/step - accuracy: 0.9725 - loss: 0.0765 - val_accuracy: 0.9333 - val_loss: 0.1683
Epoch 3/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 42s 596ms/step - accuracy: 0.9857 - loss: 0.0408 - val_accuracy: 0.9778 - val_loss: 0.0512
Epoch 4/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 43s 606ms/step - accuracy: 0.9940 - loss: 0.0206 - val_accuracy: 0.9844 - val_loss: 0.0394
Epoch 5/5
66/66 ━━━━━━━━━━━━━━━━━━━━ 43s 602ms/step - accuracy: 0.9962 - loss: 0.0129 - val_accuracy: 0.9933 - val_loss: 0.0158


In [26]:
# Save Xception Model
xception_model.save("xception_model.keras")
print("Xception training completed and model saved.")

Xception training completed and model saved.


In [27]:
print("Evaluating Xception on test data...")
xception_eval = xception_model.evaluate(test_generator)
print(f"Xception Test Accuracy: {xception_eval[1] * 100:.2f}%")

Evaluating Xception on test data...
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 136ms/step - accuracy: 0.9924 - loss: 0.0148
Xception Test Accuracy: 99.11%


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Paths to the saved models
DENSENET_MODEL_PATH = "densenet121_model.keras"
XCEPTION_MODEL_PATH = "xception_model.keras"

# Load the saved models
densenet_model = tf.keras.models.load_model(DENSENET_MODEL_PATH)
xception_model = tf.keras.models.load_model(XCEPTION_MODEL_PATH)

# Image preprocessing function
def preprocess_image(image_path, target_size=(256, 256)):
    """
    Preprocess the input image for prediction.
    Args:
        image_path (str): Path to the image.
        target_size (tuple): Target size for resizing the image.

    Returns:
        np.array: Preprocessed image array.
    """
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Prediction function
def predict_custom_image(image_path):
    """
    Predict the class of the image using both models.

    Args:
        image_path (str): Path to the image.

    Returns:
        dict: Predictions from both models.
    """
    # Preprocess the image
    img_array = preprocess_image(image_path)

    # Predict using DenseNet-121
    densenet_prediction = densenet_model.predict(img_array)
    densenet_result = "Tumor" if densenet_prediction[0][0] > 0.5 else "No Tumor"

    # Predict using Xception
    xception_prediction = xception_model.predict(img_array)
    xception_result = "Tumor" if xception_prediction[0][0] > 0.5 else "No Tumor"

    # Combine results
    predictions = {
        "DenseNet-121": {"Prediction": densenet_result, "Confidence": densenet_prediction[0][0]},
        "Xception": {"Prediction": xception_result, "Confidence": xception_prediction[0][0]}
    }
    return predictions

# Example usage
custom_image_path = r"Brain_Tumor_Detection\pred\pred3.jpg"  
results = predict_custom_image(custom_image_path)

# Display results
for model, result in results.items():
    print(f"Model: {model}")
    print(f"  Prediction: {result['Prediction']}")
    print(f"  Confidence: {result['Confidence']:.2f}")
    print()

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Model: DenseNet-121
  Prediction: No Tumor
  Confidence: 0.00

Model: Xception
  Prediction: No Tumor
  Confidence: 0.00

